In [1]:
import csv
import pandas as pd
import sys
import os
import pathlib2
import lims_sql_query as lsq

# Mouse

In [159]:
mshiny = pd.read_csv("../data/mouse_anno_table.csv",
                     usecols=["anno.sample_id", "anno.res_index_label",
                              "anno.rna_amp_pass_fail_label"])

In [160]:
#
#mshiny = mshiny[["anno.sample_id", "anno.res_index_label", "anno.rna_amp_pass_fail_label"]]
#mshiny

In [161]:
#getting all 63x_calls from LIMS
imgs_query = "Select cell.name, array_to_string(array_agg(DISTINCT tag.name), '_AND_') \
            FROM specimens cell JOIN ephys_roi_results err on err.id = cell.ephys_roi_result_id \
            JOIN specimen_tags_specimens sptagsp on sptagsp.specimen_id = cell.id \
            JOIN specimen_tags tag on tag.id = sptagsp.specimen_tag_id and tag.id in (602120185, 602122082) \
            GROUP BY cell.id \
            ORDER BY 1"
imgs_df = lsq.get_lims_dataframe(imgs_query)
#imgs_df

In [162]:
#getting driver lines for all patch-seq cells
lims_query1 = "WITH reporters AS (SELECT dg.donor_id, ARRAY_TO_STRING (ARRAY_AGG (DISTINCT g.name), ' ') AS reporters \
            FROM donors_genotypes dg JOIN genotypes g ON dg.genotype_id = g.id \
            JOIN genotype_types gt ON g.genotype_type_id = gt.id \
            WHERE gt.name ILIKE 'reporter%' \
            GROUP BY dg.donor_id ), \
            drivers AS (SELECT dg.donor_id, ARRAY_TO_STRING (ARRAY_AGG (DISTINCT g.name), ' ') AS drivers \
            FROM donors_genotypes dg \
            JOIN genotypes g ON dg.genotype_id = g.id \
            JOIN genotype_types gt ON g.genotype_type_id = gt.id \
            WHERE gt.name ILIKE 'driver%' \
            GROUP BY dg.donor_id) \
            SELECT DISTINCT s.name AS cell_name, proj.name, proj.code, rpt.reporters, drv.drivers \
            FROM specimens s JOIN projects proj ON s.project_id = proj.id \
            LEFT JOIN reporters rpt ON s.donor_id = rpt.donor_id \
            LEFT JOIN drivers drv ON s.donor_id = drv.donor_id \
            WHERE proj.code = 'mIVSCC-MET' OR proj.code = 'hIVSCC-MET'"

#OR proj.code = 'T301' OR proj.code = 'T301x'"
lims_df1 = lsq.get_lims_dataframe(lims_query1)
lims_df1 = lims_df1.drop(['name', 'reporters'], axis=1)
#lims_df1.tail()

In [163]:
#getting all patched cell containers from LIMS
lims_query2 = "SELECT s.name, s.patched_cell_container \
            FROM specimens s \
            WHERE patched_cell_container != 'None'"
lims_df2 = lsq.get_lims_dataframe(lims_query2)
#lims_df2.tail()

In [164]:
#Merging driver dataframe with patched cell container dataframe
lims_df = pd.merge(left = lims_df1, right = lims_df2, left_on = 'cell_name', right_on = 'name', how = 'right')
#lims_df

In [165]:
#Extracting date from patched cell container becusae some recording dates are missing from LIMS
lims_df['recording_date'] = lims_df['patched_cell_container'].str[5:11]
#lims_df

In [166]:
#Convertine date to datetime object
lims_df['recording_date'] = pd.to_datetime(lims_df['recording_date'], yearfirst=True, errors='coerce')
#lims_df

In [167]:
#filter out dates prior to January 1, 2018
lims_df = lims_df[lims_df['recording_date'] >= '2018-01-01']
#lims_df

In [168]:
#merge lims dataframe with 63x images dataframe together - probably should get these into a single query soon
lims = pd.merge(left = lims_df, right = imgs_df, left_on = 'name', right_on = 'name', how = 'left')
#lims

In [169]:
#merge shiny and lims into single df
smart_data = pd.merge(left = mshiny, right = lims, left_on = 'anno.sample_id', right_on = 'patched_cell_container', how = 'right')
#smart_data

In [170]:
#change header from "array_to_string" to "63x_call"
smart_data = smart_data.drop('anno.sample_id', axis=1)
smart_data = smart_data.rename(index=str, columns={"array_to_string": "63x_call"})
#smart_data

In [171]:
smart_data.sort_values(['recording_date', 'patched_cell_container'], ascending=[True, True])
mouse_smart_data = smart_data.set_index('recording_date')
#mouse_smart_data

In [172]:
#filtering out human data
mouse_smart_data = mouse_smart_data[~mouse_smart_data['name'].astype(str).str.startswith('H1')]
#mouse_smart_data

In [173]:
#Filtering out collaborators' samples
mouse_smart_data = mouse_smart_data[~mouse_smart_data['patched_cell_container'].astype(str).str.startswith('PX')]
mouse_smart_data = mouse_smart_data[~mouse_smart_data['patched_cell_container'].astype(str).str.startswith('PG')]
mouse_smart_data = mouse_smart_data[~mouse_smart_data['patched_cell_container'].astype(str).str.startswith('PH')]
mouse_smart_data = mouse_smart_data.drop('name', axis=1)
mouse_smart_data = mouse_smart_data[mouse_smart_data['code'] == 'mIVSCC-MET']
mouse_smart_data = mouse_smart_data.sort_index()
#mouse_smart_data

In [48]:
#full_path2 = home/"documents"/"github"/"SMART_goals"/"data"/"mouse_smart_data.csv"
#full_path2 = str(full_path2)
mouse_smart_data.to_csv("../data/mouse_smart_data.csv")

# Human

In [177]:
#home = pathlib2.Path.home()
#full_path = home/"documents"/"github"/"SMART_goals"/"data"/"human_anno_table.csv"
#full_path = str(full_path)
#full_path
hshiny = pd.read_csv("../data/human_anno_table.csv",
                     usecols=["anno.sample_id", "anno.res_index_label",
                              "anno.rna_amplification_pass_fail_label", "anno.roi_label"])
#hshiny

In [175]:
#hshiny = hshiny[["anno.sample_id", "anno.res_index_label",
#                 "anno.rna_amplification_pass_fail_label", "anno.roi_label"]]
#hshiny

In [178]:
hsmart_data = pd.merge(left = hshiny, right = lims, left_on = 'anno.sample_id', right_on = 'patched_cell_container', how = 'right')
#hsmart_data

In [179]:
#change header from "array_to_string" to "63x_cal"
hsmart_data = hsmart_data.drop('anno.sample_id', axis=1)
hsmart_data = hsmart_data.rename(index=str, columns={"array_to_string": "63x_call"})
#hsmart_data

In [180]:
#This is incorrect layer information
hsmart_data['layer'] = hsmart_data['anno.roi_label'].str[-1:]

In [181]:
#smart_data['recording date'] = smart_data['recording date'].dt.date
hsmart_data.sort_values(['recording_date', 'patched_cell_container'], ascending=[True, True])
human_smart_data = hsmart_data.set_index('recording_date')
human_smart_data = human_smart_data.drop('anno.roi_label', axis=1)
#human_smart_data

In [182]:
human_smart_data = human_smart_data[human_smart_data['name'].astype(str).str.startswith('H1')]
#human_smart_data

In [183]:
#Filtering out collaborators' samples
human_smart_data = human_smart_data[~human_smart_data['patched_cell_container'].astype(str).str.startswith('PX')]
human_smart_data = human_smart_data[~human_smart_data['patched_cell_container'].astype(str).str.startswith('PG')]
human_smart_data = human_smart_data[~human_smart_data['patched_cell_container'].astype(str).str.startswith('PH')]
#human_smart_data

In [58]:
#full_path2 = home/"documents"/"github"/"SMART_goals"/"data"/"human_smart_data.csv"
#full_path2 = str(full_path2)
human_smart_data.to_csv("../data/human_smart_data.csv")